## Tensorflow使用函数式API实现WideDeep模型

Wide&Deep是一个典型的多输入单输出的模型，综合利用浅层模型的记忆能力和深层模型的泛化能力：
<img src="./images/wide_deep.jpg" style="margin-left:0px; width:400px"/>

In [1]:
import tensorflow as tf
from tensorflow import keras

# 使用sklearn现成的波斯顿房价数据集
from sklearn.datasets import fetch_california_housing

# 做训练集测试集验证集拆分
from sklearn.model_selection import train_test_split

# 对数值数据归一化
from sklearn.preprocessing import StandardScaler

### 1. 下载sklearn的波士顿房价数据集

In [2]:
# 使用这个函数下载波斯顿房价数据集
housing = fetch_california_housing()

In [3]:
# data是房子的信息
housing["data"].shape

(20640, 8)

In [4]:
housing["data"][:3]

array([[ 8.32520000e+00,  4.10000000e+01,  6.98412698e+00,
         1.02380952e+00,  3.22000000e+02,  2.55555556e+00,
         3.78800000e+01, -1.22230000e+02],
       [ 8.30140000e+00,  2.10000000e+01,  6.23813708e+00,
         9.71880492e-01,  2.40100000e+03,  2.10984183e+00,
         3.78600000e+01, -1.22220000e+02],
       [ 7.25740000e+00,  5.20000000e+01,  8.28813559e+00,
         1.07344633e+00,  4.96000000e+02,  2.80225989e+00,
         3.78500000e+01, -1.22240000e+02]])

In [5]:
# target是房价
housing["target"].shape

(20640,)

In [6]:
housing["target"][:3]

array([4.526, 3.585, 3.521])

### 2. 数据分割与归一化

In [7]:
# 分割训练集、测试集、验证集
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

In [8]:
# 使用sklearn对数值数据做归一化
scaler = StandardScaler()
# 在训练集上做fit和transform
X_train = scaler.fit_transform(X_train)

# 在测试集和验证集上做transform
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

### 3. 把数据分割为Wide部分输入和Deep部分输入

In [9]:
# A是0~4共5列，B是2~7共6列
X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]

# valid和test做同样的处理
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]

In [10]:
X_train_A.shape

(11610, 5)

In [11]:
X_train_B.shape

(11610, 6)

In [12]:
# 最终的预估数据：分别取test的前三行
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

### 4. 搭建WideDeep模型

In [13]:
# 输入：wide和deep
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")

# 构造deep部分
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
hidden3 = keras.layers.Dense(30, activation="relu")(hidden2)

# 合并wide和deep
concat = keras.layers.concatenate([input_A, hidden3])

# 输出层
output = keras.layers.Dense(1, name="output")(concat)

# 构造model，注意多输入，单输出
model = keras.models.Model(inputs=[input_A, input_B], outputs=[output])

### 5. 模型编译、训练、测试、预估

In [14]:
# 模型编译
model.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=1e-3))

In [15]:
# 模型训练
model.fit((X_train_A, X_train_B), y_train, epochs=10,
    validation_data=((X_valid_A, X_valid_B), y_valid))

Epoch 1/10
363/363 [==============================] - 0s 1ms/step - loss: 2.0791 - val_loss: 0.8139
Epoch 2/10
363/363 [==============================] - 0s 1ms/step - loss: 0.7064 - val_loss: 0.6567
Epoch 3/10
363/363 [==============================] - 0s 981us/step - loss: 0.6209 - val_loss: 0.6017
Epoch 4/10
363/363 [==============================] - 0s 1ms/step - loss: 0.5827 - val_loss: 0.5713
Epoch 5/10
363/363 [==============================] - 0s 1ms/step - loss: 0.5568 - val_loss: 0.5492
Epoch 6/10
363/363 [==============================] - 1s 3ms/step - loss: 0.5365 - val_loss: 0.5319
Epoch 7/10
363/363 [==============================] - 0s 1ms/step - loss: 0.5202 - val_loss: 0.5185
Epoch 8/10
363/363 [==============================] - 0s 1ms/step - loss: 0.5085 - val_loss: 0.5108
Epoch 9/10
363/363 [==============================] - 0s 1ms/step - loss: 0.5011 - val_loss: 0.5041
Epoch 10/10
363/363 [==============================] - 0s 1ms/step - loss: 0.4942 - val_loss: 0.49

In [16]:
# 模型测试
model.evaluate((X_test_A, X_test_B), y_test)

162/162 [==============================] - 0s 3ms/step - loss: 0.4875


0.4874507486820221

In [17]:
# 模型预估
model.predict((X_new_A, X_new_B))

array([[1.7234457],
       [1.2794064],
       [0.5585957]], dtype=float32)